In [14]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np
import re
from nltk.stem import *
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import GaussianNB
from statistics import mean
from sklearn.cross_validation import KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.feature_extraction.text import CountVectorizer # for bigram

stoplist = pd.read_csv("/Users/joshuarosenberg/Dropbox/research/generality/stoplist_2.csv", header = None) # reading stoplist
data = pd.read_csv("/Users/joshuarosenberg/Dropbox/research/generality/new_data_11-17_all_rc.csv") # reading data

data.shape # dimensions
print(data.columns.values) # column names
# data.gen_or_spec = data.gen_or_spec_code # assigning gen_or_spec
data.code = data.bin5 # assigning code
data.code = data.code.astype(str)
data.text = data.Why
data.code.value_counts() # frequency of codes
data

['Lesson' 'Student_ID' 'gen_or_spec' 'Why' 'code' 'bin6' 'bin5']


,Lesson,Student_ID,gen_or_spec,Why,code,bin6,bin5
0,8.3,101402,gen,A because it can be any 2 substance to form ne...,4,4,4
1,8.3,101604,both,a for the fact of how chemical reactions happe...,4,4,4
2,8.3,101356,blank,A general way so that it explains more than on...,4,4,4
3,8.3,101452,gen,Because it will help with other problems in th...,4,4,4
4,8.3,101539,gen,Because my model shows a chemical reaction All...,4,4,4
5,8.3,101313,gen,Because we should know how things generally wo...,4,4,4
6,8.3,104114,gen,because we think that copper and acetic acid i...,4,5,4
7,8.3,104176,gen,i chose a because if you have a general model ...,4,4,4
8,8.3,104123,gen,i think it should be this way because it can e...,4,4,4
9,8.3,104166,gen,it should be this way because it can explain o...,4,4,4


In [15]:
stemmer = PorterStemmer()

proc_text = []
for i in data.index:
    tmp_text = re.sub("\d+", "", data.text[i]) # removes numbers  
    tmp_text = re.sub(r'[?|$|.|!]',r'', tmp_text) # removes puncutation
    tmp_text = tmp_text.lower() # makes text lowercase
    tmp_text = ([i for i in tmp_text.split() if i not in stoplist]) # tokenizes and removes stopwords
    tmp_text = ' '.join(tmp_text) # joins words back to stopwords
    tmp_text = stemmer.stem(tmp_text) # stems words
    proc_text.append(tmp_text)
data.proc_text = proc_text

# make term document matrix

# This 'tdm_df' function is more or less copied from the NLTK website

def tdm_df(docs, xColNames = None, **kwargs):
    vectorizer = CountVectorizer(**kwargs)
    x1 = vectorizer.fit_transform(docs)
    df = pd.DataFrame(x1.toarray().transpose(), index = vectorizer.get_feature_names())
    if xColNames is not None:
        df.columns = xColNames
    return df

pt = data.proc_text
out_tdm = tdm_df(data.proc_text) # creates term document matrix
#out_tdm.gen = data.gen # these add the dummy codes for gen or spec
#out_tdm.spec = data.spec
#out_tdm.blank = data.blank
#out_tdm.both = data.both
out_tdm = out_tdm.transpose()

bigram_vectorizer = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
X_2 = bigram_vectorizer.fit_transform(pt).toarray()
feature_names = bigram_vectorizer.get_feature_names()

codes_list = data.code.tolist()
codes_array = np.array(codes_list)
out_array = np.array(out_tdm, dtype="|S6")
out_array = np.array(out_tdm, dtype="|S6")

# classifying

gnb = GaussianNB()

X = X_2
# X = out_array
y = codes_array

kf = KFold(1155, n_folds = 1155, shuffle = True)

res_out = []
test_list = [] # for true y codes
y_test_list = [] # for test y codes

for train_index, test_index in kf:
        
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train = X_train.astype(float) # features must be numeric
    X_test = X_test.astype(float)
    
    train_fit = gnb.fit(X_train, y_train)
    test = gnb.predict(X_test)
    # res = test == y_test
    test_list.extend(test)
    y_test_list.extend(y_test)
    # res = sum(res.tolist()) / len(res.tolist())
    # res_out.append(res)
    
test_list = pd.Series(test_list)
y_test_list = pd.Series(y_test_list)

# agreement

agreement = sum(test_list == y_test_list) / float(len(y_test_list))
kappa = cohen_kappa_score(test_list, y_test_list)

print ('agreement: %s\nkappa: %s' % (agreement,kappa))

# confusion matrix:

test_list = pd.Series(test_list)
y_test_list = pd.Series(y_test_list)

x = pd.crosstab(test_list, y_test_list, rownames=['True'], colnames=['Predicted'], margins = True)
x

agreement: 0.543722943723
kappa: 0.400824514136


Predicted,0,1,2,3,4,All
True,,,,,,
0,34,5,1,0,0,40
1,1,131,25,17,53,227
2,3,89,220,14,78,404
3,3,79,28,64,53,227
4,0,33,24,21,179,257
All,41,337,298,116,363,1155
